In [34]:
#Heap stuff 
workspace()

type Node{T}
    # a node data structure. It contains two arguments: a key and a value. "key" 
    # is the used to rank the node, possibly by a heap. "val" is the value of the 
    # node from the perspective of its graph.
    key::T
    val::T
end

type Heap{T}
    # a heap data structure. It contains a list of node objects. Note that elements 
    # are never deleted from the node list; rather, a pointer called "last" is used 
    # to indicate that last good value in the arrays. The heaps can be either min or 
    # max heaps, as determined by the keyword argument "kind"
    nodelist::Array{Node{T},1}
    last::T
    kind::ASCIIString
end
Heap(T, kind) = Heap{T}([],0,kind)
 
function sink(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be sunk can be found 
    # what it does:
    #     after the base node is popped from the heap, the node in the last 
    #     position is brough into the base node, at least temporarily. The proper 
    #     position for this node can be found by "sinking" it down through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.kind == "min"
        ∘ = <=
    elseif heap.kind == "max"
        ∘ = >=
    end
    if heap.last == 0
        return
    elseif heap.last > 0
        left = heap.nodelist[2*i]
        right = heap.nodelist[2*i+1]
        if left.key ∘ right.key && left.key ∘ heap.nodelist[i].key
            switch = 2*i
        elseif right.key ∘ left.key && right.key ∘ heap.nodelist[i].key
            switch = 2*i+1
        else
            return
        end
        heap.nodelist[i], heap.nodelist[switch] = heap.nodelist[switch], heap.nodelist[i]
        return sink(heap, switch)
    end
end

function float(heap, i)
    # arguments:
    #     heap: a heap data structure
    #     i: an index describing the location in the heap (in the base array 
    #     itself) of the node to be floated can be found 
    # what it does:
    #     a new node is pushed onto the heap in the last position. This may not be 
    #     its rightful location in the heap. The proper position for this node can 
    #     be found by "floating" it up through the heap.
    # returns:
    #    return statements are for facilitation of recursion only
    if heap.kind == "min"
        ∘ = <
    elseif heap.kind == "max"
        ∘ = >
    end
    if i == 1
        return
    else
        if heap.nodelist[i].key ∘ heap.nodelist[i÷2].key
            heap.nodelist[i], heap.nodelist[i÷2] = heap.nodelist[i÷2], heap.nodelist[i]
            return float(heap, i÷2)
        end
    end
end

function extremum(heap)
     if heap.kind == "min"
        if typeof(heap) == Heap{UInt32}
            extreme = Node{UInt32}(2^32-1, 0)
        elseif typeof(heap) == Heap{Int64}
            extreme = Node{Int64}(2^32-1, 0)
        end
     elseif heap.kind == "max"
        if typeof(heap) == Heap{UInt32}
            extreme = Node{UInt32}(0, 0)
        elseif typeof(heap) == Heap{Int64}
            extreme = Node{Int64}(-2^32+1, 0)
        end
        return extreme
    end
end
        
function pop(heap)
    # arguments:
    #     heap: a heap data structure
    # what it does:
    #     removes the base node from the heap. Note that elements are never 
    #     deleted from the heap's lists, rather the last pointer is decremented
    # returns:
    #    the contents of the base node: its key and value
    outer = extremum(heap)
    if heap.last > 0
        returnable = heap.nodelist[1]
        heap.nodelist[1], heap.nodelist[heap.last] = heap.nodelist[heap.last], outer
        heap.last -= 1
        sink(heap, 1)
        return returnable
    else
        throw(error("Ain't no thing in the heap"))
    end
end

function push(heap, nuno)
    # arguments:
    #     heap: a heap data structure
    #     key: the quantity of the new node around which the heap is organized
    #     value: other quantity associated with the node but unimportant to the heap
    # what it does:
    #     adds a new node to the heap. Note that if the heap's lists are too short to 
    #     contain the new nodes children, dummy elements are added to the lists. The 
    #     keys of these dummies are the maximum allowed by the data type if a min heap, 
    #     zero if a max heap. This is done to facilitate sinking after pop.
    # returns:
    #    no returns
    outer = extremum(heap)
    if size(heap.nodelist)[1] == 0
        push!(heap.nodelist, nuno)
        heap.last = 1
    else
        heap.last += 1
        heap.nodelist[heap.last] = nuno
    end
    if 2*heap.last+1 > size(heap.nodelist)[1]
        push!(heap.nodelist,outer, outer)
    end
    float(heap, heap.last)
end

push (generic function with 1 method)

In [35]:
function reed(filename)
    graph = Dict{Int64,Dict{Int64,Int64}}()
    open(filename) do f
        for entry in eachline(f)
            adjacency = [x for x in split(entry)]
            from = parse(Int64, adjacency[1])
            graph[from] = [z[1] => z[2] 
                           for z in [map(y ->parse(Int64,y), split(x, ',')) 
                           for x in adjacency[2:end]]]
        end
    end
    return graph
end

reed (generic function with 1 method)

In [40]:
graph = reed("dijkstraData.txt")
println("ready")

ready


In [42]:
function explore(graph, heap, shortestpaths, explored)
    # arguments:
    #     heap: a heap data structure
    #     explored: a list of booleans corresponding to the visitation status of 
    #     the node corresponding to the index of the array
    # what it does:
    #     adds a new nodes to the frontier--all nodes which are adjacent to the 
    #     node at the top of the heap and that have not been previously explored.
    #     Nodes are not considered explored until they are popped off the heap. 
    #     Pops the top off the heap before it returns.
    # returns:
    #    the heap argument with new nodes to be explored and the base node popped off
    #    the updated list of explored nodes
    chekov = true
    current = Node(0,0)
    while chekov
        current = pop(heap)
        chekov = explored[current.val]
    end
    push!(shortestpaths, current)
    pathlength = current.key
    explored[current.val] = true
    for k in keys(graph[current.val])
        if !explored[k]
            push(heap, Node(pathlength + graph[current.val][k], k))
        end
    end
    return heap, shortestpaths, explored
end

function Dijkstra(graph)
    shortestpaths = []
    explored = [false for i = 1:maximum(keys(graph))]
    heap = Heap(Int64,"min")
    push(heap, Node{Int64}(0, 1))
    while !all(explored)
        heap, shortestpaths, explored = explore(graph, heap, shortestpaths, explored)
    end
    return sort(shortestpaths, by=x->x.val)
end

Dijkstra (generic function with 1 method)

In [43]:
SPs = Dijkstra(graph)
for i in [7,37,59,82,99,115,133,165,188,197]
    println(SPs[i].key)
end

2599
2610
2947
2052
2367
2399
2029
2442
2505
3068


$$ V = \frac 1 {2 n^2} \sum_i \sum_j (x_i - x_j)^2 $$

In [242]:
V1(X) = sum((X' .- X).^2)/(2*size(X)[1]^2)

V1(P)

8.25

$$ V = \frac 1 {2 n^2} \sum_i \sum_j (x_i^2 - 2 x_i x_j + x_j^2) $$

$$ V = \frac 1 {2 n^2} \sum_i \sum_j (2 x_i^2 - 2 x_i x_j) $$

$$ V = \frac 1 {n^2} \sum_i \sum_j x_i^2 - \frac 1 {n^2} \sum_i \sum_j x_i x_j $$

$$ V = \frac 1 {n^2} \sum_i n x_i^2 - \frac 1 {n^2} \sum_i \sum_j x_i x_j $$

$$ V = \frac 1 {n} \sum_i x_i^2 - \frac 1 {n^2} \sum_i \sum_j x_i x_j $$

In [245]:
V2(X) = (size(X)[1]*sum((X.^2)) - sum(X'.*X))/(size(X)[1]^2)

V2(P)

8.25

$$ V = \frac 1 {n} \sum_i x_i^2 - \frac 1 {n^2} \sum_i  x_i \sum_j x_j $$

$$ V = \frac 1 {n} \sum_i x_i^2 - \frac 1 {n^2} n \bar x n \bar x $$

$$ V = \overline {x^2}  -  {\overline x}^2 $$

$$ V[x] = E [ {x^2}]  -  E[ x]^2 $$

In [247]:
V3(X) = mean(X.^2) - mean(X)^2

V3(P)

8.25

$$ V = \frac 1 {2 n^2} \sum_i \sum_j (x_i - x_j)^2 $$

$$ V = \frac 1 {2 n^2} \sum_i \sum_j (x_i^2 - 2 x_i x_j +x_j^2) $$

$$ V = \frac 1 {2 n^2} \sum_i (\sum_j  x_i^2 - 2 \sum_j  x_i x_j + \sum_j x_j^2) $$

$$ V = \frac 1 {2 n^2} \sum_i (n  x_i^2 - 2 n \overline x  x_i + n \overline{x^2}) $$

But we already proved that $\displaystyle V = \overline{x^2} -{\overline x}^2$. Therefore

$$ V = \frac 1 {2 n^2} \sum_i (n  x_i^2 - 2 n \overline x  x_i + n V + n {\overline x}^2) $$

$$ V = \frac 1 {2 n^2} \left( n  \sum_i x_i^2 - 2 n \sum_i \overline x  x_i + n \sum_i V + n \sum_i {\overline x}^2 \right) $$

$$ V = \frac 1 {2 n^2} \left( n  \sum_i x_i^2 - 2 n \sum_i \overline x  x_i + n^2  V + n \sum_i {\overline x}^2 \right)$$

$$ V = \frac V 2 + \frac 1 {2 n} \sum_i \left( x_i^2 - 2 \overline x  x_i + {\overline x}^2 \right)$$

$$ \frac V 2 = \frac 1 {2 n} \sum_i \left( x_i -  {\overline x} \right)^2$$

$$ V = \frac 1 {n} \sum_i \left( x_i -  {\overline x} \right)^2$$

$$ V = \frac 1 {n} \sum_i \left( x_i -  {\overline x} \right)^2$$

$$ V = \frac 1 {n} \sum_i \left( x_i -  \frac 1 n \sum_j x_j \right)^2$$

$$ V = \frac 1 {n} \sum_i \left( \frac 1 n \sum_j x_i -  \frac 1 n \sum_j x_j \right)^2$$

$$ V = \frac 1 {n^3} \sum_i \left( \sum_j \left(x_i - x_j \right)\right)^2$$

In [260]:
sum([sum([pi - pj for pj in P])^2 for pi in P])/size(P)[1]^3

8.25

Somehow $ \displaystyle V = \frac 1 {2 n^2} \sum_i \sum_j \left(x_i - x_j \right)^2 = \frac 1 {n^3} \sum_i \left( \sum_j \left(x_i - x_j \right)\right)^2$ but I cannot prove it